In [1]:
import ibis
from pyspark.sql import SparkSession
import spark_df_profiling

In [2]:

session = SparkSession.builder.getOrCreate()
ipsc = ibis.pyspark.connect(session)


22/09/26 23:34:33 WARN Utils: Your hostname, tmps-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.62 instead (on interface en0)
22/09/26 23:34:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 23:34:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
session.sparkContext._conf.getAll()

[('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'),
 ('spark.sql.warehouse.dir',
  'file:/Users/mjboothaus/code/github/mjboothaus/sparky/notebooks/spark-warehouse'),
 ('spark.driver.port', '55193'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1664199275837'),
 ('spark.a

In [4]:
# df_rdd = session.read.csv("../data/sample_big_mortgage_file.csv.gz", header=True, sep="|")

df_rdd = session.read.csv("../data/2020_lar.txt.gz", header=True, sep="|")

In [5]:
# Try making dataframe a table and query with SQL

df_rdd.createTempView("tmp_df_table")

22/09/26 23:34:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
ipsc

In [7]:
ipsc.list_databases()

['default']

In [8]:
ipsc.list_tables()


['tmp_df_table']

In [9]:
my_table = ipsc.table("tmp_df_table")

In [10]:
for col in my_table.columns:
    if col[0] == "d":
        print(col)

derived_msa_md
derived_loan_product_type
derived_dwelling_category
derived_ethnicity
derived_race
derived_sex
discount_points
debt_to_income_ratio
denial_reason_1
denial_reason_2
denial_reason_3
denial_reason_4


In [11]:
query = "SELECT * FROM tmp_df_table LIMIT 5"

In [12]:
query = "SELECT DISTINCT derived_sex FROM tmp_df_table"

In [13]:
df1 = session.sql(query)

In [14]:
df1.show()

+-----------------+
|      derived_sex|
+-----------------+
|Sex Not Available|
|           Female|
|            Joint|
|             Male|
+-----------------+



In [15]:
# df_rdd.count()

In [16]:
### Now the Ibis approach

ibis.options.interactive = True

In [17]:
ipsc.current_database

'default'

In [18]:
ipsc.exists_table("tmp_df_table")

/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_7939/3364586496.py:1: FutureWarning: `exists_table` is deprecated as of v2.0; use `name in client.list_tables()`
  ipsc.exists_table("tmp_df_table")


True

In [19]:
ipsc.table("tmp_df_table").columns

['activity_year',
 'lei',
 'derived_msa_md',
 'state_code',
 'county_code',
 'census_tract',
 'conforming_loan_limit',
 'derived_loan_product_type',
 'derived_dwelling_category',
 'derived_ethnicity',
 'derived_race',
 'derived_sex',
 'action_taken',
 'purchaser_type',
 'preapproval',
 'loan_type',
 'loan_purpose',
 'lien_status',
 'reverse_mortgage',
 'open_end_line_of_credit',
 'business_or_commercial_purpose',
 'loan_amount',
 'combined_loan_to_value_ratio',
 'interest_rate',
 'rate_spread',
 'hoepa_status',
 'total_loan_costs',
 'total_points_and_fees',
 'origination_charges',
 'discount_points',
 'lender_credits',
 'loan_term',
 'prepayment_penalty_term',
 'intro_rate_period',
 'negative_amortization',
 'interest_only_payment',
 'balloon_payment',
 'other_nonamortizing_features',
 'property_value',
 'construction_method',
 'occupancy_type',
 'manufactured_home_secured_property_type',
 'manufactured_home_land_property_interest',
 'total_units',
 'multifamily_affordable_units',
 'in

In [20]:
t = ipsc.table("tmp_df_table")

In [21]:
expr = t.count()

In [22]:
# expr

In [23]:
# ibis.show_sql(expr) <<--- this doesn't seem to work 

In [24]:
## Profiling

In [25]:
# df_sample = df_rdd.sample(fraction=0.1, seed=42).cache()

In [26]:
# df_sample.count()

In [27]:
df_rdd_cache = df_rdd.cache()
df_rdd_cache.persist()

22/09/26 23:35:55 WARN CacheManager: Asked to cache already cached data.


DataFrame[activity_year: string, lei: string, derived_msa_md: string, state_code: string, county_code: string, census_tract: string, conforming_loan_limit: string, derived_loan_product_type: string, derived_dwelling_category: string, derived_ethnicity: string, derived_race: string, derived_sex: string, action_taken: string, purchaser_type: string, preapproval: string, loan_type: string, loan_purpose: string, lien_status: string, reverse_mortgage: string, open_end_line_of_credit: string, business_or_commercial_purpose: string, loan_amount: string, combined_loan_to_value_ratio: string, interest_rate: string, rate_spread: string, hoepa_status: string, total_loan_costs: string, total_points_and_fees: string, origination_charges: string, discount_points: string, lender_credits: string, loan_term: string, prepayment_penalty_term: string, intro_rate_period: string, negative_amortization: string, interest_only_payment: string, balloon_payment: string, other_nonamortizing_features: string, prop

In [28]:
profile = spark_df_profiling.ProfileReport(df_rdd_cache)
profile.to_file(outputfile="../data/profile-report.html")

22/09/26 23:36:29 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)
22/09/26 23:36:29 WARN BlockManager: Persisting block rdd_32_0 to disk instead.


22/09/27 00:20:12 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


22/09/27 00:20:16 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


22/09/27 00:20:37 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


22/09/27 00:21:02 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


...

/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:418: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_categorical_1d(df, column))


22/09/27 10:12:28 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


22/09/27 10:12:39 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


22/09/27 10:12:51 WARN MemoryStore: Not enough space to cache rdd_32_0 in memory! (computed 286.2 MiB so far)


/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:418: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_categorical_1d(df, column))
